In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
from matplotlib import pyplot
import os
import keras
from keras.preprocessing.image import load_img, img_to_array
from skimage.io import imread
from PIL import Image #PIL is no longer maintained and it is not compatible with Python 3.X. To use Pillow instead
from numpy import load
from os import listdir
from numpy import savez_compressed
from numpy import asarray
from google.colab import files
from keras import Sequential
from keras.layers import *
from keras import regularizers
from keras.optimizers import Adam, SGD

In [ ]:
#to do:
# *from the training folder for kaggle, select the kaggle images with every class. Put them in a folder by class
# *make a npz file with the images for every class
#run the baseline GAN for every class
#store fake images of every class in a separate folder
#decide how many fake images from every class I want to use for the sclassification. Probably 20% of the real images of every class, in order to maintain the proportion of every class
#Create a folder with real + fake images for all the classes
#create a npz file with the images from all the folders. Make sure that the images are shuffled. For ex, real images should be mixed with synthetic images and images from different classes should be shuffled.
#create a copy of the classifier file to use the new npz file
#do classification with the classifier that I already have 

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [ ]:
#load images from npz
def load_real_samples(filename):
  # load dataset
  data = load(filename)
  # extract numpy array
  X = data['arr_0']
  # convert from ints to floats
  X = X.astype('float32')
  return X

In [ ]:
def discriminator(img_shape):
  depth = 64
  dropout = 0.3
  lrL2 = 1e-7

  D = Sequential()
  D.add(keras.Input(shape=(128,128,3)))

  D.add(Conv2D(depth*1, 5, strides=2, padding='same', kernel_regularizer=regularizers.l2(lrL2)))
  D.add(BatchNormalization())
  D.add(ReLU())
  D.add(Dropout(dropout))
 
  D.add(Conv2D(depth*2, 5, strides=2, padding='same', kernel_regularizer=regularizers.l2(lrL2)))
  D.add(BatchNormalization())
  D.add(ReLU())
  D.add(Dropout(dropout))

  D.add(Conv2D(depth*4, 5, strides=2, padding='same', kernel_regularizer=regularizers.l2(lrL2)))
  D.add(BatchNormalization())
  D.add(ReLU())
  D.add(Dropout(dropout))

  D.add(Conv2D(depth*8, 5, strides=1, padding='same', kernel_regularizer=regularizers.l2(lrL2)))
  D.add(BatchNormalization())
  D.add(ReLU())
  D.add(Dropout(dropout))

  D.add(Flatten())
  D.add(Dense(1, activation="sigmoid"))

  return D

my_disc = discriminator((128, 128, 3))
my_disc.summary()

In [ ]:
def generator():

    G = Sequential()
    depth = 64*4
    img_rows = 128
    dim = int(img_rows / 8 )
    dropout = 0.3
    lrL2 = 1e-7

    G.add(keras.Input(shape=(100,1)))
    G.add(Dense(512))
    G.add(BatchNormalization())
    G.add(ReLU())
    G.add(Dropout(dropout))
    G.add(Reshape((dim, dim, 200)))

    G.add(UpSampling2D(size=(2, 2)))

    G.add(Conv2D(int(depth), 5, padding='same', kernel_regularizer=regularizers.l2(lrL2)))
    G.add(BatchNormalization())
    G.add(ReLU())
    G.add(Dropout(dropout))
    G.add(Conv2D(int(depth/2), 5, padding='same', kernel_regularizer=regularizers.l2(lrL2)))
    G.add(ReLU())
    G.add(Dropout(dropout))

    G.add(UpSampling2D(size=(2, 2)))

    G.add(Conv2D(int(depth/4), 5, padding='same', kernel_regularizer=regularizers.l2(lrL2)))
    G.add(BatchNormalization())
    G.add(ReLU())
    G.add(Dropout(dropout))
    G.add(Conv2D(int(depth/8), 5, padding='same', kernel_regularizer=regularizers.l2(lrL2)))
    G.add(ReLU())
    G.add(Dropout(dropout))

    G.add(UpSampling2D(size=(2, 2)))

    G.add(Conv2D(int(depth/16), 5, padding='same', kernel_regularizer=regularizers.l2(lrL2)))
    G.add(BatchNormalization())
    G.add(ReLU())
    G.add(Dropout(dropout))
    G.add(Conv2D(int(depth/32), 5, padding='same', kernel_regularizer=regularizers.l2(lrL2)))
    G.add(ReLU())
    G.add(Dropout(dropout))

    G.add(Conv2D(3, 5, padding='same', kernel_regularizer=regularizers.l2(lrL2)))
    G.add(ReLU())
    return G

gen_model = generator()
gen_model.summary()

In [ ]:
#generate synthetic data using the generator
def generate_fake_samples_using_generator(g_model, noise_dim, n_samples):
  
	# generate random data points of size noise_dim
  flattened_noise_dim = noise_dim[0]
  x = np.random.randn(n_samples * flattened_noise_dim)
  # reshape into a batch of inputs for the network
  x_data = x.reshape(n_samples, noise_dim[0], noise_dim[1])
	
  # push data through the generator model
  x = g_model.predict(x_data)

  # set class labels to 0 for artificial data
  y = np.zeros((n_samples, 1))

  return x, y

#create GAN architecture
def gan(g_model , d_model):
  # disable weight update on GAN model
  d_model.trainable = False

  # connect the generator with the discriminator
  model = tf.keras.models.Sequential()
  model.add(g_model)
  model.add(d_model)
  # compile model
  lrAdv = 1e-4
  decayAdv = 1e-6
  optimizer_adver = Adam(lrAdv)#, decay = decayAdv)

  model.compile(loss = 'binary_crossentropy', optimizer = optimizer_adver)
  return model

In [ ]:
def get_real_samples (dataset, n_samples):
  #randomly generate indices to extract from training set
  indicies = np.random.randint(0, dataset.shape[0], n_samples)
  #extract batch of slected images
  X = dataset[indicies]
  #set class label to 1
  y = np.ones((n_samples, 1))
  return X, y

def generate_random_vectors(noise_dim, batch_size):
  #generate random numbers from a normal distribution
  x = np.random.randn(batch_size * noise_dim[0] * noise_dim[1])

  #reshape into a batch of randomized vectors
  #(each vector will be inputted to the generator and will output an image)
  x_data = x.reshape(batch_size, noise_dim[0], noise_dim[1])
  
  return x_data

# create a line plot of loss for the gan and save to file
def plot_history(d_real_loss, d_fake_loss, g_loss):
  # plot loss
  pyplot.subplot(1, 1, 1)
  pyplot.plot(d_real_loss, label='d-real')
  pyplot.plot(d_fake_loss, label='d-fake')
  pyplot.plot(g_loss, label='gen')
  pyplot.legend()
  pyplot.show()
  pyplot.close()

def plot_history_accuracy(d_real_acc, d_fake_acc):
  pyplot.subplot(1, 1, 1)
  pyplot.plot(d_real_acc, label='acc-real')
  pyplot.plot(d_fake_acc, label='acc-fake')
  pyplot.legend()
  pyplot.show()
  pyplot.close()

In [ ]:
#train the gan
import cv2
OUTPUT_DIR = "/content/gdrive/My Drive/img_retina_Kaggle_class_4_state_of_art_GAN"

#train the generator and discriminator
def train(g_model, d_model, gan_model, dataset, noise_dim, epochs, batch_size):
  drealLoss = []
  dfakeLoss = []
  gLoss = []
  accuracies_real = []
  accuracies_fake = []
  inx = []
  counter = 0
  batches = int(dataset.shape[0] / batch_size)

  #  for each epoch
  for i in range(epochs):
    print("Epoch: " + str(i))
    #for each batch
    for j in range(batches):
      
      #obtain real image data
      X_real, y_real = get_real_samples(dataset, batch_size)

      #generate artificial data
      X_fake, y_fake = generate_fake_samples_using_generator(g_model, noise_dim, batch_size)
    
      d_loss_real, d_accuracy_real = d_model.train_on_batch(X_real, y_real)
      d_loss_fake, d_accuracy_fake = d_model.train_on_batch(X_fake, y_fake)

      #X, y = np.vstack((X_real, X_fake)), np.vstack((y_real, y_fake))
      #d_loss, d_accuracy = d_model.train_on_batch(X, y)

      # generate random vectors of noise (this becomes input the GAN model)
      X_gan = generate_random_vectors(noise_dim, batch_size)

      # assign all GAN output data vectors a class label of 1
      y_gan = np.ones((batch_size, 1))

      # train the generator
      gan_loss = gan_model.train_on_batch(X_gan , y_gan)

      print("Batch: " + str(j) + " generator loss: " + str(gan_loss) + ", discriminator real loss: " + str(d_loss_real)  + ", discriminator fake loss: " + str(d_loss_fake))
      #print("Batch: " + str(j) + " generator loss: " + str(gan_loss) + ", discriminator loss: " + str(d_loss))

      drealLoss.append(d_loss_real)
      dfakeLoss.append(d_loss_fake)
      gLoss.append(gan_loss)
      accuracies_real.append(d_accuracy_real)
      accuracies_fake.append(d_accuracy_fake)
        
    if ((i+1) % 10) == 0:

      if(os.path.exists(OUTPUT_DIR + "/" + str(i)) == False):
        os.mkdir(OUTPUT_DIR + "/" + str(i))

      #g_model.save("generatormodelweights" + str(i) + ".h5")
      #disc_model.save("discmodelweights" + str(i) + ".h5") 

      noise_dim = (100, 1)
      n_samples = 165
      x,y = generate_fake_samples_using_generator(g_model, noise_dim, n_samples)
      for m in range(n_samples):
        this_image = x[m]
        this_image = this_image.astype(np.uint8)   
        image_name = str(i) + "_" + str(j)+"_"+str(m)
        im2 = Image.fromarray(this_image)
        im2.save("{}/{}.jpeg".format(OUTPUT_DIR + "/" + str(i),image_name))

    plot_history(drealLoss, dfakeLoss, gLoss)
    plot_history_accuracy(accuracies_real, accuracies_fake)

    #files.download("ganmodelweights" + str(i) + ".h5")
    #files.download("discmodelweights" + str(i) + ".h5")    

In [ ]:
noise_dim = (100, 1)
shape=(128,128,3)

disc_model =  discriminator(shape)
lrDisc = 1e-4
decayDisc = 1e-5 #2e-5


optimizer_disc = SGD(lrDisc)#, decay=decayDisc) #, momentum = 0.9) #, decay=decayDisc)

disc_model.compile(loss = 'binary_crossentropy', optimizer=optimizer_disc, metrics=['accuracy'])

generator_model = generator()
gan_model = gan(generator_model, disc_model)

#load image data
dataset = load_real_samples("/content/gdrive/My Drive/MSc. project/npz files conference/Kaggle_training_images_in_order_preprocessed_class4_normalized3.npz")

#train model
epochs=500
batch_size= 4 

train(generator_model, disc_model, gan_model, dataset, noise_dim, epochs, batch_size)

In [ ]:
#print(len(os.listdir('/content/gdrive/My Drive/Kaggle/training preprocessed')))
#images = np.load("/content/gdrive/My Drive/MSc. project/npz files conference/real + fake/Kaggle_training_images_class4_real_plus_fake_epoch_19.npz")["arr_0"]

In [ ]:
#import matplotlib.pyplot as plt
#plt.ion()
#plt.figure()
#plt.imshow(images[2])